scraper.ipynb


In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import time

base_url = "http://books.toscrape.com/catalogue/"
books = []

for page in range(1, 4):   # scrape first 3 pages
    url = f"http://books.toscrape.com/catalogue/page-{page}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    items = soup.find_all("article", class_="product_pod")

    for item in items:
        # Basic info from listing page
        title = item.h3.a["title"]
        price = item.find("p", class_="price_color").text

        #  Rating extraction
        rating_class = item.find("p", class_="star-rating")["class"][1]
        rating = rating_class   # e.g., One, Two, Three...

        #  Go to book detail page
        book_link = item.h3.a["href"]
        book_url = base_url + book_link

        book_page = requests.get(book_url)
        book_soup = BeautifulSoup(book_page.text, "html.parser")

        # Stock availability
        stock = book_soup.find("p", class_="instock availability").text.strip()

        # Category extraction (breadcrumb navigation)
        breadcrumb = book_soup.find("ul", class_="breadcrumb").find_all("li")
        category = breadcrumb[2].text.strip()

        # \ Save data
        books.append([title, price, rating, stock, category])

        # delay
        time.sleep(1)

#  Save to CSV
with open("books.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "Price", "Rating", "Stock", "Category"])
    writer.writerows(books)

print("Data saved to books.csv")


Data saved to books.csv


In [11]:
import pandas as pd
df =pd.read_csv("/content/books.csv")

In [12]:
df.head()

,Title,Price,Rating,Stock,Category
0,A Light in the Attic,Â£51.77,Three,In stock (22 available),Poetry
1,Tipping the Velvet,Â£53.74,One,In stock (20 available),Historical Fiction
2,Soumission,Â£50.10,One,In stock (20 available),Fiction
3,Sharp Objects,Â£47.82,Four,In stock (20 available),Mystery
4,Sapiens: A Brief History of Humankind,Â£54.23,Five,In stock (20 available),History
